In [26]:
import pandas as pd

talks = pd.read_csv('../../tedtalks/data/tedtalks2018.csv')
speakers = pd.read_csv('../../speakersGenderTest.csv', encoding='latin-1')
new = pd.read_csv('../../tedtalks/data/tedtalks2018.csv')

In [27]:
print(speakers.shape)
print(talks.shape)

(1800, 10)
(2656, 12)


In [28]:
speakers.head()

,Name,Occupation,ShortDescription,LongDescription,Gender,MaleScore,FemaleScore,NonBinaryScore,Gender + hand codes,Unnamed: 9
0,Abha Dawesar,Novelist,Abha Dawesar writes to make sense of the world...,Why you should listen\rAbha Dawesar began her ...,female,0,10,0,female,NaN
1,Abigail Washburn,Clawhammer banjo player,Abigail Washburn pairs venerable folk elements...,Why you should listen\rIf American old-time mu...,female,0,5,0,female,NaN
2,Achenyo Idachaba,Green entrepreneur,"Achenyo Idachaba is the head of MitiMeth, a Ni...","Why you should listen\rIn 2009, Achenyo Idacha...",female,0,4,0,female,NaN
3,Aditi Gupta,"Social entrepreneur, co-founder of Menstrupedia",Aditi Gupta uses storytelling and art to educa...,Why you should listen\rAditi Gupta is a social...,female,0,6,0,female,NaN
4,Aditi Shankardass,Neuroscientist,Aditi Shankardass is pioneering the use of EEG...,Why you should listen\rAditi Shankardass is a ...,female,0,3,1,female,NaN


In [29]:
speakers.rename(columns={'Name':'speaker_name'}, inplace=True)


In [35]:
speakers.drop(speakers.columns[len(speakers.columns)-1], axis=1, inplace=True)

#https://stackoverflow.com/questions/20517650/how-to-delete-the-last-column-of-data-of-a-pandas-dataframe

In [36]:
speakers.head()

,speaker_name,Occupation,ShortDescription,LongDescription,Gender,MaleScore,FemaleScore,NonBinaryScore,Gender + hand codes
0,Abha Dawesar,Novelist,Abha Dawesar writes to make sense of the world...,Why you should listen\rAbha Dawesar began her ...,female,0,10,0,female
1,Abigail Washburn,Clawhammer banjo player,Abigail Washburn pairs venerable folk elements...,Why you should listen\rIf American old-time mu...,female,0,5,0,female
2,Achenyo Idachaba,Green entrepreneur,"Achenyo Idachaba is the head of MitiMeth, a Ni...","Why you should listen\rIn 2009, Achenyo Idacha...",female,0,4,0,female
3,Aditi Gupta,"Social entrepreneur, co-founder of Menstrupedia",Aditi Gupta uses storytelling and art to educa...,Why you should listen\rAditi Gupta is a social...,female,0,6,0,female
4,Aditi Shankardass,Neuroscientist,Aditi Shankardass is pioneering the use of EEG...,Why you should listen\rAditi Shankardass is a ...,female,0,3,1,female


In [55]:
result = pd.merge(talks, speakers, on="speaker_name", how="left", indicator=True)
#https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/

In [56]:
result.head()

,rowID,Talk_ID,public_url,speaker_name,headline,description,event,duration,published,tags,...,text,Occupation,ShortDescription,LongDescription,Gender,MaleScore,FemaleScore,NonBinaryScore,Gender + hand codes,_merge
0,0,1,https://www.ted.com/talks/al_gore_on_averting_...,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",...,"Thank you so much, Chris. And it's truly a g...",Climate advocate,Nobel Laureate Al Gore focused the world_ã_s a...,Why you should listen\rFormer Vice President A...,male,5.0,0.0,0.0,male,both
1,1,7,https://www.ted.com/talks/david_pogue_says_sim...,David Pogue,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",...,"(Music: ""The Sound of Silence,"" Simon & Garf...",Technology columnist,David Pogue is the personal technology columni...,Why you should listen\rWhich cell phone to cho...,male,5.0,0.0,0.0,male,both
2,2,53,https://www.ted.com/talks/majora_carter_s_tale...,Majora Carter,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",...,If you're here today — and I'm very happy th...,Activist for environmental justice,Majora Carter redefined the field of environme...,Why you should listen\rMajora Carter is a visi...,female,1.0,11.0,0.0,female,both
3,3,66,https://www.ted.com/talks/ken_robinson_says_sc...,Ken Robinson,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",...,Good morning. How are you? (Laughter) ...,Author/educator,Creativity expert Sir Ken Robinson challenges ...,Why you should listen\rWhy don't we get the be...,male,8.0,0.0,2.0,male,both
4,4,92,https://www.ted.com/talks/hans_rosling_shows_t...,Hans Rosling,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",...,"About 10 years ago, I took on the task to te...",Global health expert; data visionary,"In Hans Rosling_ã_s hands, data sings. Global ...",Why you should listen\rEven the most worldly a...,male,13.0,0.0,2.0,male,both


In [53]:
print(result.shape)
print(talks.shape)
print(speakers.shape)

(2664, 20)
(2656, 12)
(1800, 9)


In [58]:
result.loc[result['_merge'] == "right_only"]
#https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas

,rowID,Talk_ID,public_url,speaker_name,headline,description,event,duration,published,tags,...,text,Occupation,ShortDescription,LongDescription,Gender,MaleScore,FemaleScore,NonBinaryScore,Gender + hand codes,_merge


In [59]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2664 entries, 0 to 2663
Data columns (total 21 columns):
rowID                  2664 non-null int64
Talk_ID                2664 non-null int64
public_url             2664 non-null object
speaker_name           2662 non-null object
headline               2664 non-null object
description            2664 non-null object
event                  2664 non-null object
duration               2664 non-null object
published              2664 non-null object
tags                   2664 non-null object
views                  2664 non-null int64
text                   2664 non-null object
Occupation             2083 non-null object
ShortDescription       2086 non-null object
LongDescription        2087 non-null object
Gender                 2087 non-null object
MaleScore              2087 non-null float64
FemaleScore            2087 non-null float64
NonBinaryScore         2087 non-null float64
Gender + hand codes    2087 non-null object
_merge       

In [50]:
speakers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 9 columns):
speaker_name           1800 non-null object
Occupation             1796 non-null object
ShortDescription       1799 non-null object
LongDescription        1800 non-null object
Gender                 1800 non-null object
MaleScore              1800 non-null int64
FemaleScore            1800 non-null int64
NonBinaryScore         1800 non-null int64
Gender + hand codes    1800 non-null object
dtypes: int64(3), object(6)
memory usage: 126.6+ KB


In [62]:
# Can't run just yet
# result['MaleScore'] = result['MaleScore'].astype(int)
# https://stackoverflow.com/questions/41590884/change-data-type-of-a-specific-column-of-a-pandas-dataframe

ValueError: Cannot convert NA to integer

In [65]:
result.loc[result['MaleScore'].isnull()].shape

(577, 21)